In [ ]:
# *TRAIN THE SPAM DETECTION MODEL (Backend)*

# Import required libraries
import re
import string
import pickle
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# 1. Load dataset
df = pd.read_csv('/content/spam dataset.csv', encoding='latin-1')[['v1', 'v2']]
df.columns = ['label', 'message']

# 2. Encode labels (spam = 1, ham = 0)
df["label"] = df["label"].map({"spam": 1, "ham": 0})

# 3. Remove duplicates
df = df.drop_duplicates()

# 4. Preprocess text
def preprocess_text(text):
    text = text.lower()  # Convert to lowercase
    text = re.sub(f"[{string.punctuation}]", "", text)  # Remove punctuation
    return text

df["cleaned_message"] = df["message"].apply(preprocess_text)

# 5. Vectorize text
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df["cleaned_message"])
y = df["label"]

# 6. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 7. Train Naive Bayes model
model = MultinomialNB()
model.fit(X_train, y_train)

# 8. Evaluate
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"✅ Model Accuracy: {accuracy * 100:.2f}%")

# 9. Save the model and vectorizer
with open("spam_model.pkl", "wb") as model_file:
    pickle.dump(model, model_file)

with open("vectorizer.pkl", "wb") as vec_file:
    pickle.dump(vectorizer, vec_file)

print("✅ Model and vectorizer saved successfully.")


✅ Model Accuracy: 97.87%
✅ Model and vectorizer saved successfully.


In [ ]:
# 🛠 Create the app.py file inside Colab

%%writefile app.py
import streamlit as st
import pickle
import re
import string

# 1. Load model and vectorizer
model = pickle.load(open("spam_model.pkl", "rb"))
vectorizer = pickle.load(open("vectorizer.pkl", "rb"))

# 2. Function to preprocess input text
def preprocess_text(text):
    text = text.lower()
    text = re.sub(f"[{string.punctuation}]", "", text)
    return text

# 3. Streamlit App
st.set_page_config(page_title="Spam Detection App", page_icon="📩")

st.title("📩 Spam Detection App")
st.write("Enter your message below and I'll tell you if it's Spam or Not Spam!")

user_input = st.text_area("Enter a message:")

if st.button("Predict"):
    if user_input.strip() == "":
        st.warning("⚠️ Please enter a message.")
    else:
        cleaned_input = preprocess_text(user_input)
        vectorized_input = vectorizer.transform([cleaned_input])
        prediction = model.predict(vectorized_input)[0]

        if prediction == 1:
            st.error("🚨 Spam Message Detected!")
        else:
            st.success("✅ Not a Spam Message!")


Writing app.py
